In [192]:
#import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
import numpy as np
from reference_converters import *

In [193]:
# loading data 
compound_table = pd.read_parquet('compound_table.parquet')
compound_table.sort_values('id', ascending=True, inplace=True)
compound_table.rename(columns={'id': 'compound_id'}, inplace=True)
electrochem_table = pd.read_parquet('electrochem_table.parquet')


In [194]:
# creating merged table
combined_table = compound_table.merge(electrochem_table, how='right', on='compound_id')
combined_table.dropna(subset=['name',], axis=0, inplace=True)
combined_table= combined_table[['compound_id', 'name', 'formula', 'recyclable', 'type', 'compound_family',
       'id', 'redox_potential_V', 'reference_electrode',
       'electrolyte', 'solvent', 'pH', 'used_in_photocat', 'used_in_rfbs', 'used_as_hcarrier', 'used_in_DSSC', 'measurement_ref'
       ]]

In [195]:
# data cleaning

# checking category binary
combined_table.loc[combined_table.used_in_photocat!=1, 'used_in_photocat'] = 0

# splitting aqueous non-aqueous
combined_table['aqueous'] = 0
combined_table.loc[combined_table.solvent=='water', 'aqueous']=1
combined_table.loc[combined_table.solvent=='aqueous', 'aqueous']=1

# altering quinone label to hydroquinone for aqueous media
combined_table.loc[(combined_table.aqueous==1) & (combined_table.compound_family=='quinone'), 'compound_family'] = 'hydroquinone'


In [196]:
print(f"Table contains data for {combined_table.compound_id.nunique()} compounds:")
print(f"compound_family: {combined_table.compound_family.nunique()} distinct")
print(f"type: {combined_table.type.nunique()} distinct")
print(f"solvent: {combined_table.solvent.nunique()} distinct")
print(f"{combined_table.loc[combined_table.aqueous==1, 'compound_id'].nunique()} compounds with redox potentials measured in aqueous media")

Table contains data for 51 compounds:
compound_family: 14 distinct
type: 3 distinct
solvent: 12 distinct
18 compounds with redox potentials measured in aqueous media


In [197]:
display(combined_table[['compound_family', 'type', 'compound_id', 'id']].groupby(['type', 'compound_family']).nunique())

compound_id  id
type              compound_family                    
electron acceptor dialkylquinoxaline            1   2
                  pyridinium                    1   1
                  viologen                      3   5
electron donor    R-NADH                        2   2
                  TEMPO                         2   2
                  acridine                      1   1
                  amine                        11  11
                  benzimidazole                 9  10
                  dialkoxybenzene               3   4
                  disulfide forming             8   8
                  ferrocene                     3   3
                  hydroquinone                  3   3
                  quinone                       1   1
photosensitizer   organometallic                4  18

In [198]:
# setting aqueous data to one reference redox potentials
print(combined_table.loc[combined_table.aqueous==1,'reference_electrode'].unique())
print(combined_table.loc[combined_table.aqueous==1,'pH'].unique())

['SCE' 'NHE' 'SHE' 'Ag|AgCl']
[nan 7.  0.  0.7]


In [199]:
print(combined_table.loc[combined_table.aqueous!=1,'reference_electrode'].unique())

['SCE' 'SHE' 'Li|Li+' 'Fc/Fc+' 'NHE' 'Ag|AgCl']


In [200]:
print(combined_table.loc[combined_table.reference_electrode=='Li|Li+', 'solvent'].unique())

['EC/PC/EMC' 'PC' 'EC/EMC' 'PEC']


In [201]:
print(combined_table.loc[combined_table.aqueous!=1, 'solvent'].unique())

['acetonitrile' 'THF' 'EC/PC/EMC' 'PC' 'EC/EMC' 'PEC' 'MeCN' 'DMF' 'DMSO'
 'gamma-butyrolactone']


In [202]:
display(combined_table.sort_values('compound_id'))

,compound_id,name,formula,recyclable,type,compound_family,id,redox_potential_V,reference_electrode,electrolyte,solvent,pH,used_in_photocat,used_in_rfbs,used_as_hcarrier,used_in_DSSC,measurement_ref,aqueous
0,1,triethylamine,(C2H5)3N,0.0,electron donor,amine,1,0.69,SCE,,water,NaN,1.0,0.0,0.0,0.0,https://doi.org/10.1016/j.crci.2015.11.026,1
1,2,triethanolamine,(HOCH2CH2)3N,0.0,electron donor,amine,2,1.07,NHE,pH7 phosphate buffer,water,7.0,1.0,0.0,0.0,0.0,https://doi.org/10.1039/c3cp55023k,1
31,4,4-dimethylaminotoluene (DMT),CH3C6H4N(CH3)2,0.0,electron donor,amine,32,0.70,SCE,TBAClO4,MeCN,NaN,1.0,0.0,0.0,0.0,https://doi.org/10.1016/S0022-0728(70)80304-7,0
4,5,"1,3-dimethyl-2-phenylbenzimidazoline (BIH)",C15H16N2,0.0,electron donor,benzimidazole,5,0.33,SCE,Et4NClO4,acetonitrile,NaN,1.0,0.0,0.0,0.0,doi:10.1016/j.tet.2006.03.061,0
50,5,"1,3-dimethyl-2-phenylbenzimidazoline (BIH)",C15H16N2,0.0,electron donor,benzimidazole,51,-0.44,Fc/Fc+,,DMSO,NaN,1.0,0.0,0.0,0.0,DOI: 10.1039/C9CC00928K,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,53,"1,1,2,2-tetrapiperidinoethene (TPiE)",C22H40N4,0.0,electron donor,amine,69,-1.06,Fc/Fc+,TBAPF6,DMF,NaN,0.0,0.0,0.0,0.0,DOI: 10.1021/jacs.1c10932,0
66,54,"4-(1,2,2-trimorpholin-4-ylethene)morpholine (TME)",C18H32N4O4,0.0,electron donor,amine,70,-0.85,Fc/Fc+,TBAPF6,DMF,NaN,0.0,0.0,0.0,0.0,DOI: 10.1021/jacs.1c10932,0
67,55,tetrakis(N-pyrrolidinyl)ethylene (TPyE),C18H32N4,0.0,electron donor,amine,71,-1.32,Fc/Fc+,TBAPF6,DMF,NaN,0.0,0.0,0.0,0.0,DOI: 10.1021/jacs.1c10932,0
68,56,"1-[1,2,2-tris(azepan-1-yl)ethenyl]azepane (TAzE)",C26H48N4,0.0,electron donor,amine,72,-1.09,Fc/Fc+,TBAPF6,DMF,NaN,0.0,0.0,0.0,0.0,DOI: 10.1021/jacs.1c10932,0


In [203]:
# converting potentials to vs. SCE for aqueous measurments
combined_table.loc[combined_table.aqueous==1,:] = combined_table.loc[combined_table.aqueous==1,:].apply(lambda row: aqueous_potentials_to_SCE(row), axis=1)
# converting potentials to vs. Fc/Fc+ for nonaqueous measurments
combined_table.loc[combined_table.aqueous!=1,:] = combined_table.loc[combined_table.aqueous!=1,:].apply(lambda row: nonaqueous_potentials_to_Fc(row), axis=1)

In [204]:
# split photosensitizers into a different table
ps_table = combined_table.loc[combined_table.type=='photosensitizer', :].copy(deep=True)
rsd_table = combined_table.loc[combined_table.type!='photosensitizer', :].copy(deep=True)

In [205]:
display(rsd_table[['compound_family', 'compound_id', 'aqueous', 'used_in_photocat', 'used_in_rfbs', 'used_as_hcarrier', 'used_in_DSSC']].groupby(['aqueous', 'compound_family','used_in_photocat', 'used_in_rfbs', 'used_as_hcarrier', 'used_in_DSSC']).nunique())

compound_id
aqueous compound_family    used_in_photocat used_in_rfbs used_as_hcarrier used_in_DSSC             
0       acridine           1.0              0.0          0.0              0.0                     1
        amine              0.0              0.0          0.0              0.0                     5
                                                         1.0              0.0                     1
                           1.0              0.0          0.0              0.0                     2
        benzimidazole      1.0              0.0          0.0              0.0                     9
        dialkoxybenzene    0.0              1.0          0.0              0.0                     3
        dialkylquinoxaline 0.0              1.0          0.0              0.0                     1
        disulfide forming  0.0              0.0          0.0              1.0                     5
                           1.0              0.0          0.0              0.0                     3
        quinone            0.0              0.0          1.0              0.0                     1
        viologen           0.0              1.0          0.0              0.0                     1
1       R-NADH             1.0              0.0          0.0              0.0                     2
        TEMPO              0.0              1.0          0.0              0.0                     2
        amine              1.0              0.0          0.0              0.0                     3
        ferrocene          0.0              1.0          0.0              0.0                     3
        hydroquinone       0.0              0.0          1.0              0.0                     2
                                            1.0          1.0              0.0                     1
        pyridinium         0.0              1.0          0.0              0.0                     1
        viologen           0.0              1.0          0.0              0.0                     3

In [206]:
def label_function(row):
    label_tuple = (row.at['used_in_photocat'], row.at['recyclable'], row.at['used_in_rfbs'], row.at['used_as_hcarrier'], row.at['used_in_DSSC'])
    match label_tuple:
        case (1, 1, 0, 0, 0):
            label_text = 'recyclable donor'
        case (1, 0, 0, 0, _):
            label_text = 'sacrificial donor'
        case (0, _, 1, 0, 0):
            label_text = 'RFB redox couple'
        case (0, _, _, 1, _):
            label_text = 'hydrogen carrier'
        case (0, _, 0, 0, 1):
            label_text = 'DSSC redox couple'
        case _:
            label_text = 'other source'
    row.at['label'] = label_text
    return row

In [207]:
rsd_table['label'] = ''
rsd_table = rsd_table.apply(lambda row: label_function(row), axis=1)

In [208]:
rsd_table.label.unique()

array(['sacrificial donor', 'recyclable donor', 'RFB redox couple',
       'hydrogen carrier', 'other source', 'DSSC redox couple'],
      dtype=object)

In [209]:
ps_table['redox_event'] = ''
ps_table.loc[ps_table.id.isin([33, 48]), 'redox_event'] = 'Ered'
ps_table.loc[ps_table.id.isin([34, 50, 55, 42]), 'redox_event'] = 'Ered*'
ps_table.loc[ps_table.id.isin([35, 47, 54, 39]), 'redox_event'] = 'Eox'
ps_table.loc[ps_table.id.isin([36, 49, 56]), 'redox_event'] = 'Eox*'
#display(ps_table)

In [210]:
# function for assigning plot markers and colours
def marker_array_generator(labels_array, marker_dict):
    marker_array = []
    for lbl in labels_array:
        marker_array.append(marker_dict[lbl])
    return marker_array
symbol_dict = {'sacrificial donor': 100, 'recyclable donor': 2, '':102, 'RFB redox couple': 0, 'hydrogen carrier': 4, 'DSSC redox couple': 1, 'other source':102}
color_dict = {'sacrificial donor': '#2E4F4F', 'recyclable donor': '#FC4F00', '':'#0A4D68', 'RFB redox couple': '#B71375', 'hydrogen carrier': '#088395', 'DSSC redox couple': '#00FFCA', 'other source':'#0A4D68'}
#'#05BFDB'

In [211]:
#-----------------------------------------------------------!plotcode
# initiate plot
aqueous = go.Figure()

# make a trace per label
# prepare variables
compound_x = rsd_table.loc[rsd_table.aqueous==1,'compound_family'].unique().tolist()
compound_labels = rsd_table.loc[rsd_table.aqueous==1,'label'].unique().tolist()
for c_label in compound_labels:
    x_var = rsd_table.loc[(rsd_table.aqueous==1) & (rsd_table.label==c_label),'compound_family']
    y_var = rsd_table.loc[(rsd_table.aqueous==1) & (rsd_table.label==c_label),'redox_potential_V']
    c_var = rsd_table.loc[(rsd_table.aqueous==1) & (rsd_table.label==c_label),'label']
    
    # list for holding hover label info
    hover_text1 = zip(rsd_table.loc[(rsd_table.aqueous==1) & (rsd_table.label==c_label),'compound_id'].to_list(),
                       rsd_table.loc[(rsd_table.aqueous==1) & (rsd_table.label==c_label),'name'].to_list())
    hover_text1 = [f"Compound {txt1}: {txt2}" for (txt1, txt2) in hover_text1]

    # make the trace
    aqueous.add_trace(go.Scatter(
        x= x_var, 
        y=y_var,
        mode="markers", 
        showlegend=True,
        name=c_var.iat[0],
        marker_symbol = marker_array_generator(c_var, symbol_dict), 
        marker_color = marker_array_generator(c_var, color_dict),
        text = hover_text1,
        hovertemplate="%{text}<br>"+"%{y:.2f} V vs. SCE"
                        ))

# format axes, legend, title, and marker size
aqueous.update_yaxes(title=dict(text='Oxidation Potential vs. SCE', font=dict(size=16, family='Arial'),),
                        range=[2,-1], linecolor='#283149', mirror=True,
                        title_standoff=8,)
aqueous.update_xaxes(title='',  linecolor='#283149', mirror=True)
aqueous.update_layout(title=dict(text="aqueous media (mostly neutral pH)", font=dict(size=16,),), 
                         title_font_family='Arial', 
                         title_x=0.1, title_y=0.82,
                         font=dict(size=12, family="Arial"),
                         template='plotly_white', 
                         width=900, height=600, 
                         legend=dict(title='', ))
aqueous.update_traces(marker=dict(size=8,))

# adding lines for rubpy potentials
id_list = ps_table.loc[(ps_table.compound_id==36)&(ps_table.aqueous==1),:].id.to_list()
id_list = [idnum for idnum in id_list if 'red*' in ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item() or ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item()=='Eox']
for idnum in id_list:
    aqueous.add_hline(y=ps_table.loc[ps_table.id==idnum, 'redox_potential_V'].values.item(),
                      line = dict(dash='dot', color='grey'),
                      annotation_text=f"[Ru(bpy)3] {ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item()}",
                      annotation_position="bottom left"
                     )
# order x-axis to help labelling
aqueous.update_xaxes(categoryorder='array', categoryarray=['R-NADH', 'amine', 'hydroquinone', 'ferrocene', 'TEMPO', 'viologen', 'pyridinium'])

# draw boxes
aqueous.add_shape(type="rect", xref="paper", yref="paper",
                     x0=0.76, y0=0.72,
                     x1=0.96, y1=0.97,
                     line=dict(color='grey', width=2,)
                     )
aqueous.add_shape(type="rect", xref="paper", yref="paper",
                     x0=0.48, y0=0.35,
                     x1=0.68, y1=0.5,
                     line=dict(color='grey', width=2,)
                     )

# annotate boxes
aqueous.add_trace(go.Scatter(
    x=['viologen', ],
    y=[-0.05,],
    mode="text",
    name="Text",
    text=["RFB anolytes", ],
    textposition="middle right",
    showlegend=False,
))
aqueous.add_trace(go.Scatter(
    x=['ferrocene', ],
    y=[0.47,],
    mode="text",
    name="Text",
    text=["RFB catholytes", ],
    textposition="top right",
    showlegend=False,
))

# create pH labels
pH_vars = rsd_table.loc[(rsd_table.aqueous==1) & (rsd_table.pH!=7.0),:]
pH_x = pH_vars.loc[pH_vars.pH.notna(),'compound_family'].to_list()
pH_y = pH_vars.loc[pH_vars.pH.notna(),'redox_potential_V'].to_list()
pH_var = pH_vars.loc[pH_vars.pH.notna(),'pH'].to_list()

# add pH annotation
for pHpoint in range(0, len(pH_var)):
    aqueous.add_annotation(
        x=pH_x[pHpoint],
        y=pH_y[pHpoint],
        text=pH_var[pHpoint],
        #textposition="top right",
        showarrow=True,
    )

aqueous.show()


In [212]:
# list for holding hover label info
hover_text = zip(rsd_table.loc[rsd_table.aqueous!=1,'compound_id'].to_list(), rsd_table.loc[rsd_table.aqueous!=1,'name'].to_list())
hover_text = [f"Compound {txt1}: {txt2}" for (txt1, txt2) in hover_text]

#-----------------------------------------------------------!plotcode
# initiate plot
nonaqueous = go.Figure()

# make a trace per label
# get info
compound_x = rsd_table.loc[rsd_table.aqueous!=1,'compound_family'].unique().tolist()
compound_labels = rsd_table.loc[rsd_table.aqueous!=1,'label'].unique().tolist()
for c_label in compound_labels:
    x_var = rsd_table.loc[(rsd_table.aqueous!=1) & (rsd_table.label==c_label),'compound_family']
    y_var = rsd_table.loc[(rsd_table.aqueous!=1) & (rsd_table.label==c_label),'redox_potential_V']
    c_var = rsd_table.loc[(rsd_table.aqueous!=1) & (rsd_table.label==c_label),'label']

    # list for holding hover label info
    hover_text1 = zip(rsd_table.loc[(rsd_table.aqueous!=1) & (rsd_table.label==c_label),'compound_id'].to_list(),
                       rsd_table.loc[(rsd_table.aqueous!=1) & (rsd_table.label==c_label),'name'].to_list())
    hover_text1 = [f"Compound {txt1}: {txt2}" for (txt1, txt2) in hover_text1]

    # make the trace
    nonaqueous.add_trace(go.Scatter(
        x= x_var, 
        y=y_var,
        mode="markers", 
        showlegend=True,
        name=c_label,
        marker_symbol = marker_array_generator(c_var, symbol_dict), 
        marker_color = marker_array_generator(c_var, color_dict),
        #color_discrete_sequence= [, , , , , , '#2E4F4F'],
        text = hover_text1,
        hovertemplate="%{text}<br>"+"%{y:.2f} V vs. Fc/Fc+"
                        ))



# format axes, legend and title
nonaqueous.update_yaxes(title=dict(text='Oxidation Potential vs. Fc/Fc+', font=dict(size=16, family='Arial'),),
                        range=[2,-1.5], linecolor='#283149', mirror=True,
                        title_standoff=8,)
nonaqueous.update_xaxes(title='',  linecolor='#283149', mirror=True)
nonaqueous.update_layout(title=dict(text="non-aqueous media", font=dict(size=16,),), 
                         title_font_family='Arial', 
                         title_x=0.1, title_y=0.82,
                         font=dict(size=12, family="Arial"),
                         template='plotly_white', 
                         width=900, height=600, 
                         legend=dict(title='', ))
nonaqueous.update_traces(marker=dict(size=8,))

# add horizantal lines for dyes
id_list = ps_table.loc[(ps_table.compound_id==36)&(ps_table.aqueous!=1),:].id.to_list()
id_list = [idnum for idnum in id_list if 'red*' in ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item() or ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item()=='Eox']
for idnum in id_list:
    nonaqueous.add_hline(y=ps_table.loc[ps_table.id==idnum, 'redox_potential_V'].values.item(),
                      line = dict(dash='dot', color='grey'),
                      annotation_text=f"[Ru(bpy)3] {ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item()}",
                      annotation_position="bottom left"
                     )
id_list = ps_table.loc[(ps_table.compound_id==44)&(ps_table.aqueous!=1),:].id.to_list()
id_list = [idnum for idnum in id_list if ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item()=='Ered*' or ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item()=='Eox']
for idnum in id_list:
    nonaqueous.add_hline(y=ps_table.loc[ps_table.id==idnum, 'redox_potential_V'].values.item(),
                      line = dict(dash='dot', color='grey'),
                      annotation_text=f"[Ir(ppy)3] {ps_table.loc[ps_table.id==idnum, 'redox_event'].values.item()}",
                      annotation_position="bottom left"
                     )
    
# order x-axis to help labelling
nonaqueous.update_xaxes(categoryorder='array', 
                        categoryarray=['benzimidazole', 'acridine', 'amine', 'disulfide forming', 'quinone', 'dialkoxybenzene', 'viologen', 'dialkylquinoxaline'])

# create boxes
nonaqueous.add_shape(type="rect", xref="paper", yref="paper",
                     x0=0.28, y0=0.78,
                     x1=0.34, y1=0.99,
                     line=dict(color='grey', width=2,)
                     )
nonaqueous.add_shape(type="rect", xref="paper", yref="paper",
                     x0=0.66, y0=0.36,
                     x1=0.71, y1=0.46,
                     line=dict(color='grey', width=2,)
                     )
nonaqueous.add_shape(type="rect", xref="paper", yref="paper",
                     x0=0.78, y0=0.68,
                     x1=0.96, y1=0.95,
                     line=dict(color='grey', width=2,)
                     )

# annotate boxes
nonaqueous.add_trace(go.Scatter(
    x=['amine', 'dialkoxybenzene'],
    y=[-0.6, 0.8,],
    mode="text",
    name="Text",
    text=["TDAE analogues", "RFB catholyte", ],
    textposition="middle center",
    showlegend=False,
))
nonaqueous.add_trace(go.Scatter(
    x=['viologen'],
    y=[-0.3],
    mode="text",
    name="Text",
    text=["RFB anolytes"],
    textposition="middle right",
    showlegend=False,
))

nonaqueous.show()

In [213]:
display(ps_table)

,compound_id,name,formula,recyclable,type,compound_family,id,redox_potential_V,reference_electrode,electrolyte,solvent,pH,used_in_photocat,used_in_rfbs,used_as_hcarrier,used_in_DSSC,measurement_ref,aqueous,redox_event
32,36,"[Ru(bpy)3]2+ Tris(2,2-bipyridyl)ruthenium(II)...",C30H36Cl2N6O6Ru,1.0,photosensitizer,organometallic,33,-1.038,SCE,,water,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,1,Ered
33,36,"[Ru(bpy)3]2+ Tris(2,2-bipyridyl)ruthenium(II)...",C30H36Cl2N6O6Ru,1.0,photosensitizer,organometallic,34,1.082,SCE,,water,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,1,Ered*
34,36,"[Ru(bpy)3]2+ Tris(2,2-bipyridyl)ruthenium(II)...",C30H36Cl2N6O6Ru,1.0,photosensitizer,organometallic,35,1.502,SCE,,water,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,1,Eox
35,36,"[Ru(bpy)3]2+ Tris(2,2-bipyridyl)ruthenium(II)...",C30H36Cl2N6O6Ru,1.0,photosensitizer,organometallic,36,-0.618,SCE,,water,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,1,Eox*
36,37,[Ir(tpy)3]3+,,1.0,photosensitizer,organometallic,37,-1.080,Fc/Fc+,,acetonitrile,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,0,
37,37,[Ir(tpy)3]3+,,1.0,photosensitizer,organometallic,38,-0.610,Fc/Fc+,,acetonitrile,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,0,
38,37,[Ir(tpy)3]3+,,1.0,photosensitizer,organometallic,39,1.630,Fc/Fc+,,acetonitrile,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,0,Eox
39,38,[Cu(dpp)]+,,1.0,photosensitizer,organometallic,40,0.380,Fc/Fc+,,acetonitrile,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,0,
40,38,[Cu(dpp)]+,,1.0,photosensitizer,organometallic,41,-1.990,Fc/Fc+,,acetonitrile,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,0,
41,38,[Cu(dpp)]+,,1.0,photosensitizer,organometallic,42,-1.420,Fc/Fc+,,acetonitrile,NaN,1.0,0.0,0.0,0.0,Balzani book page 199,0,Ered*


In [214]:
display(rsd_table.loc[rsd_table.compound_id>51,:])

,compound_id,name,formula,recyclable,type,compound_family,id,redox_potential_V,reference_electrode,electrolyte,solvent,pH,used_in_photocat,used_in_rfbs,used_as_hcarrier,used_in_DSSC,measurement_ref,aqueous,label
64,52,tetrakis(dimethylamino)ethylene (TDAE),C10H24N4,0.0,electron donor,amine,68,-1.090,Fc/Fc+,unspecified,acetonitrile,NaN,1.0,0.0,0.0,0.0,DOI: 10.1039/d0pp00127a,0,sacrificial donor
65,53,"1,1,2,2-tetrapiperidinoethene (TPiE)",C22H40N4,0.0,electron donor,amine,69,-1.060,Fc/Fc+,TBAPF6,DMF,NaN,0.0,0.0,0.0,0.0,DOI: 10.1021/jacs.1c10932,0,other source
66,54,"4-(1,2,2-trimorpholin-4-ylethene)morpholine (TME)",C18H32N4O4,0.0,electron donor,amine,70,-0.850,Fc/Fc+,TBAPF6,DMF,NaN,0.0,0.0,0.0,0.0,DOI: 10.1021/jacs.1c10932,0,other source
67,55,tetrakis(N-pyrrolidinyl)ethylene (TPyE),C18H32N4,0.0,electron donor,amine,71,-1.320,Fc/Fc+,TBAPF6,DMF,NaN,0.0,0.0,0.0,0.0,DOI: 10.1021/jacs.1c10932,0,other source
68,56,"1-[1,2,2-tris(azepan-1-yl)ethenyl]azepane (TAzE)",C26H48N4,0.0,electron donor,amine,72,-1.090,Fc/Fc+,TBAPF6,DMF,NaN,0.0,0.0,0.0,0.0,DOI: 10.1021/jacs.1c10932,0,other source
71,57,1-methyl-4-benzoylpyridinium,C13H12NO,1.0,electron acceptor,pyridinium,75,-0.805,SCE,NaCl,aqueous,7.0,0.0,1.0,0.0,0.0,DOI: 10.1021/acs.jpcc.7b06247,1,RFB redox couple


In [222]:
#creating SI table 1
si_1 = rsd_table.loc[rsd_table.aqueous==1,['compound_id', 
                                           'name', 
                                           'formula',
                                           'type', 
                                           'label',
                                           'compound_family', 
                                           'redox_potential_V',
                                           'reference_electrode',
                                           'electrolyte',
                                           'solvent',
                                           'pH',
                                           'measurement_ref',
                                           ]].copy(deep=True)
si_1.head(5)
si_1.to_csv("si1_table_aqueous_redox_potentials.csv", ",")

In [223]:
#creating SI table 2
si_2 = rsd_table.loc[rsd_table.aqueous!=1,['compound_id', 
                                           'name', 
                                           'formula', 
                                           'type', 
                                           'label',
                                           'compound_family', 
                                           'redox_potential_V',
                                           'reference_electrode',
                                           'electrolyte',
                                           'solvent',
                                           'measurement_ref',
                                           ]].copy(deep=True)
si_2.head(5)
si_2.to_csv("si2_table_nonaqueous_redox_potentials.csv", ",")